In [ ]:
pip install --upgrade kiteconnect

In [ ]:
import logging
from kiteconnect import KiteConnect
logging.basicConfig(level=logging.DEBUG)
import os
import logging
from datetime import datetime, time, timedelta # Added timedelta for precise time checks
from kiteconnect import exceptions as kc_exceptions # For specic error handling
import time
import pytz
import threading
TIMEZONE = 'Asia/Kolkata'

kite = KiteConnect(api_key="xyz")

In [ ]:
kite.login_url()

'https://kite.zerodha.com/connect/login?api_key=t68lihmtgkv55ee3&v=3'

In [ ]:
data = kite.genearate_session("Request token", api_secret="abc")

In [ ]:
kite.set_access_token(data["access_token"])

In [ ]:
        # {
        #     "tradingsymbol": "COLAB", "exchange": "BSE", "transaction_type": "SELL",
        #     "quantity": 2, "price": 37.56, "product": "CNC", "order_type": "LIMIT",
        #     "validity": "DAY", "variety": "regular",
        # },

Order placing & Time

In [ ]:
TARGET_HOUR = 15
TARGET_MINUTE = 1
TARGET_SECOND = 0
TARGET_MICROSECOND = 0

def execute_my_action():
    current_time_action = datetime.now(pytz.timezone(TIMEZONE))
    print(f"\n--- ACTION EXECUTED! ---")
    print(f"Time of execution: {current_time_action.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]} (IST)")
    print("Executing your Zerodha trading logic here...")

    ORDERS_TO_PLACE = [
        {
            "tradingsymbol": "COLAB", "exchange": "BSE", "transaction_type": "SELL",
            "quantity": 1, "price": 37.56, "product": "CNC", "order_type": "LIMIT",
            "validity": "DAY", "variety": "regular",
        },
        {
            "tradingsymbol": "COLAB", "exchange": "BSE", "transaction_type": "SELL",
            "quantity": 2, "price": 37.56, "product": "CNC", "order_type": "LIMIT",
            "validity": "DAY", "variety": "regular",
        },
        {
            "tradingsymbol": "COLAB", "exchange": "BSE", "transaction_type": "SELL",
            "quantity": 3, "price": 37.56, "product": "CNC", "order_type": "LIMIT",
            "validity": "DAY", "variety": "regular",
        },
        {
            "tradingsymbol": "COLAB", "exchange": "BSE", "transaction_type": "SELL",
            "quantity": 1, "price": 37.56, "product": "CNC", "order_type": "LIMIT",
            "validity": "DAY", "variety": "regular",
        },
        {
            "tradingsymbol": "COLAB", "exchange": "BSE", "transaction_type": "SELL",
            "quantity": 2, "price": 37.56, "product": "CNC", "order_type": "LIMIT",
            "validity": "DAY", "variety": "regular",
        },
    ]

    threads = []
    if not ORDERS_TO_PLACE:
        logging.warning("No orders defined in ORDERS_TO_PLACE. Skipping order placement.")
        return

    for i, order_data in enumerate(ORDERS_TO_PLACE):
        # Create a copy of the order_data dictionary because we will modify it with .pop()
        # and we don't want to affect the original list.
        current_order_params = order_data.copy()

        # Extract 'variety' as a positional argument first
        # This is the key change to resolve the TypeError
        try:
            variety_arg = current_order_params.pop("variety")
        except KeyError:
            logging.error(f"Order {i+1} is missing the required 'variety' key. Skipping this order.")
            continue # Skip to the next order

        tradingsymbol_log = current_order_params.get('tradingsymbol', 'N/A')
        quantity_log = current_order_params.get('quantity', 'N/A')

        # Define a small helper function or lambda for the thread target
        # This function will capture the specific order details for each thread
        def thread_target(var, params_dict, ts_log, qty_log, order_num):
            try:
                order_id = kite.place_order(var, **params_dict)
                logging.info(f"Order {order_num} placed for {ts_log} (Qty: {qty_log}, Variety: {var}). ID is: {order_id}")
            except Exception as e:
                logging.error(f"Order {order_num} placement failed for {ts_log} (Qty: {qty_log}): {e}")

        # Create and start the thread
        thread = threading.Thread(
            target=thread_target,
            args=(variety_arg, current_order_params, tradingsymbol_log, quantity_log, i+1)
        )
        threads.append(thread)
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    logging.info("All order placement attempts have been initiated and processed.")


local_tz = pytz.timezone(TIMEZONE)
current_time = datetime.now(local_tz)

target_time_today = current_time.replace(
    hour=TARGET_HOUR,
    minute=TARGET_MINUTE,
    second=TARGET_SECOND,
    microsecond=TARGET_MICROSECOND
)

if current_time >= target_time_today:
    print(f"Current time ({current_time.strftime('%H:%M:%S.%f')[:-3]} IST) is already past or exactly the target time for today ({target_time_today.strftime('%H:%M:%S.%f')[:-3]} IST).")
    print("Action not executed as the target time has passed.")
else:
    time_to_wait_seconds = (target_time_today - current_time).total_seconds()

    print(f"Current time (IST): {current_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}")
    print(f"Target execution time (IST): {target_time_today.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}")
    print(f"Waiting for {time_to_wait_seconds:.3f} seconds until {TARGET_HOUR:02}:{TARGET_MINUTE:02}:{TARGET_SECOND:02}.{TARGET_MICROSECOND:06} IST...")

    while datetime.now(local_tz) < target_time_today:
        time.sleep(0.001)
    execute_my_action()

Current time (IST): 2025-07-07 15:00:45.730
Target execution time (IST): 2025-07-07 15:01:00.000
Waiting for 14.270 seconds until 15:01:00.000000 IST...

--- ACTION EXECUTED! ---
Time of execution: 2025-07-07 15:01:00.000 (IST)
Executing your Zerodha trading logic here...
